# 用描述符来改写需要复用的@property方法

@property的缺点是不便于复用，受它修饰的方法，无法为同一个类中的其他属性所复用，而且与之无关的类也无法复用这些方法。

**示例：**验证学生的家庭作业成绩都处在0-100。

In [1]:
class Homework(object):
    def __init__(self):
        self._grade = 0

    @property
    def grade(self):
        return self._grade

    @grade.setter
    def grade(self, value):
        if not (0 <= value <= 100):
            raise ValueError('Grade must be between 0 and 100')
        self._grade = value

In [2]:
galileo = Homework()
galileo.grade = 95
print(galileo.grade)

95


**需求变更：**要把这套验证逻辑放在考试成绩上面，而考试成绩又是由多个科目的小成绩组成，每一科都要单独计分。

In [3]:
class Exam(object):
    def __init__(self):
        self._writing_grade = 0
        self._math_grade = 0

    @staticmethod
    def _check_grade(value):
        if not (0 <= value <= 100):
            raise ValueError('Grade must be between 0 and 100')
            
    @property
    def writing_grade(self):
        return self._writing_grade

    @writing_grade.setter
    def writing_grade(self, value):
        self._check_grade(value)
        self._writing_grade = value

    @property
    def math_grade(self):
        return self._math_grade

    @math_grade.setter
    def math_grade(self, value):
        self._check_grade(value)
        self._math_grade = value

In [4]:
galileo = Exam()
galileo.writing_grade = 85
galileo.math_grade = 99
print('Writing: %5r' % galileo.writing_grade)
print('Math:    %5r' % galileo.math_grade)

Writing:    85
Math:       99


**缺点：**每添加一项科目，就要重复编写一次@property方法，而且要把相关验证逻辑也重做一遍。

**改进：**采用Python的描述符，\_\_get\_\_和\_\_set\_\_方法，复用分数验证功能。

In [5]:
class Grade(object):
    def __get__(*args, **kwargs):
        pass

    def __set__(*args, **kwargs):
        pass

In [6]:
class Exam(object):
    # Class attributes
    math_grade = Grade()
    writing_grade = Grade()
    science_grade = Grade()

为属性赋值：

In [7]:
exam = Exam()
exam.writing_grade = 40

上述代码会被转译为：

In [8]:
Exam.__dict__['writing_grade'].__set__(exam, 40)

而获取属性：

In [9]:
print(exam.writing_grade)

None


上述代码会被转译为：

In [10]:
print(Exam.__dict__['writing_grade'].__get__(exam, Exam))

None


## 第一次改写：采用Python描述符改写代码

In [11]:
class Grade(object):
    def __init__(self):
        self._value = 0

    def __get__(self, instance, instance_type):
        return self._value

    def __set__(self, instance, value):
        if not (0 <= value <= 100):
            raise ValueError('Grade must be between 0 and 100')
        self._value = value

In [12]:
class Exam(object):
    math_grade = Grade()
    writing_grade = Grade()
    science_grade = Grade()

In [13]:
first_exam = Exam()
first_exam.writing_grade = 82
first_exam.science_grade = 99
print('Writing', first_exam.writing_grade)
print('Science', first_exam.science_grade)

Writing 82
Science 99


In [14]:
second_exam = Exam()
second_exam.writing_grade = 75
print('Second', second_exam.writing_grade, 'is right')
print('First ', first_exam.writing_grade, 'is wrong')

Second 75 is right
First  75 is wrong


**错误原因：**当程序定义Exam类的时候，它会把Grade实例构建好，以后创建Exam实例时，就不再构建Grade了。

## 第二次改写：把每个Exam实例所对应的值记录到Grade中。

In [15]:
class Grade(object):
    def __init__(self):
        self._values = {}

    def __get__(self, instance, instance_type):
        if instance is None: return self
        return self._values.get(instance, 0)

    def __set__(self, instance, value):
        if not (0 <= value <= 100):
            raise ValueError('Grade must be between 0 and 100')
        self._values[instance] = value

**问题：**容易泄漏内存。字典中保存了实例的引用，导致实例的引用计数无法降为0，使垃圾收集器无法将其回收。

## 第三次改写：使用weakref模块中的WeakKeyDictionary的特殊字典。

**好处：**在运行期间系统发现这种字典所持有的引用，是整个程序里指向Exam实例的最后一份引用，那么系统就会自动将该实例从字典的键中移除。

In [16]:
from weakref import WeakKeyDictionary

class Grade(object):
    def __init__(self):
        self._values = WeakKeyDictionary()
    def __get__(self, instance, instance_type):
        if instance is None: return self
        return self._values.get(instance, 0)

    def __set__(self, instance, value):
        if not (0 <= value <= 100):
            raise ValueError('Grade must be between 0 and 100')
        self._values[instance] = value

In [17]:
class Exam(object):
    math_grade = Grade()
    writing_grade = Grade()
    science_grade = Grade()

In [18]:
first_exam = Exam()
first_exam.writing_grade = 82
second_exam = Exam()
second_exam.writing_grade = 75
print('First ', first_exam.writing_grade, 'is right')
print('Second', second_exam.writing_grade, 'is right')

First  82 is right
Second 75 is right
